In [1]:
# !pip install transformers==4.46.2 datasets==3.1.0 numpy==1.26.4 sklearn-pandas==2.2.0 torch==2.5.1+cu121

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
# from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import numpy as np
import datasets
import time
import wandb
import random
# import os
# from fvcore.nn import FlopCountAnalysis
# from torch.utils.data import DataLoader
import torch

key_file = r'C:\Development\TactitalTensorsFinalProject\WANDB_API_KEY.txt' 

with open(key_file, "r") as f:
    api_key = f.read().strip()

# Log into WandB with the API key
wandb.login(key=api_key)

# Initialize WandB (no need to manually set the WANDB_API_KEY env variable again)


c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torc

True

The WANDB_API_KEY is nessesary to use the Trainer class from HuggingFace and is what's used in the example notebook. However, I think we may all have to put in our own api key secret into google colab for it to work.

In [2]:
# from google.colab import userdata
# os.environ["WANDB_API_KEY"] = userdata.get('WANDB_API_KEY')


In [3]:
langs = ['java', 'python', 'pharo']

# langs = ['java'] # Using Java as the only language for testing purposes.
# langs = ['python']
# langs = ['pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = datasets.load_dataset('NLBSE/nlbse25-code-comment-classification')

In [4]:
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

In [5]:
ds['java_test'][0]

{'index': 5,
 'class': 'AbstractContractGetFileStatusTest.java',
 'comment_sentence': 'accept everything.',
 'partition': 1,
 'combo': 'accept everything. | AbstractContractGetFileStatusTest.java',
 'labels': [0, 0, 1, 0, 0, 0, 0]}

In [ ]:
model_name = 'distilbert/distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

The next few functions are to preprocess the trainng and validation sets, and to let the Trainer class to evaluate how well the trainng is going after each epoch.

In [7]:
# To tokenize the text in the 'combo' column of the training dataset of each language.

def tokenize_dataset(examples):
  return tokenizer(examples['combo'], truncation=True, padding="max_length", max_length=128)

In [8]:
# Tokenizes the text 'combo' column of the dataset, changes the values of the
# labels column to float instead of int, and sets to format of the lables column
# to torch tesnsors, which is required by the Trainer class.

def preprocess_dataset(input_dataset):
  processed_dataset = input_dataset.map(tokenize_dataset, batched=True, load_from_cache_file=False)
  processed_dataset = processed_dataset.cast_column("labels", datasets.features.Sequence(datasets.features.Value("float32")))
  processed_dataset.set_format('pt')
  return processed_dataset

Evaluating the predictions.

Below is non-functional code I still haven't finnished. We need to implement a way for the logits returned from the model to be converted into binary (0 - 1) predictions for the labels, and for those labels to be compaired to the ground truth labels of the testing datasets for each language. The competition also wants us to complete this step using profiling that they have set up, and which is partially shown below. The link to the reference notebook for the competition is here. Also, another notebook that is fairly close to what I believe we are trying to do can be found here_for_multi_label_text_classification.ipynb).

In [ ]:
# scores must be global so that it can later be referenced for printing the results
scores = pd.DataFrame(columns=['comparison', 'model', 'lan', 'cat', 'precision', 'recall', 'f1'])

def compute_metrics(eval_pred, lang, categories, mod_name, comp):
    # global scores
    # scores.drop(scores[scores['lan'] == lang].index, inplace=True)

    predictions, true_labels = eval_pred
    predictions = (predictions > 0.5).astype(int)
    num_classes = len(categories)

    metrics_list = []

    for i in range(num_classes):
        y_pred = predictions[:, i]
        y_true = true_labels[:, i]

        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        metrics_list.append({
            'model': mod_name,
            'lan': lang,
            'comparison': comp,
            'cat': categories[i],
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

    temp_scores = pd.DataFrame([m for m in metrics_list if m['model'] == mod_name])
    scores = pd.concat([scores, temp_scores], ignore_index=True)

    avg_f1 = temp_scores['f1'].mean()
    avg_precision = temp_scores['precision'].mean()
    avg_recall = temp_scores['recall'].mean()

    return {'f1': avg_f1, 'precision': avg_precision, 'recall': avg_recall}

In [10]:
def measure_runtime_and_flops(trainer, dataset):
    flops = 0
    runtime = 0

    for i in range(10):
        for batch in dataset:
            inputs = {key: val.unsqueeze(0).to(trainer.model.device) for key, val in batch.items() if key in ['input_ids', 'attention_mask']}
            with torch.profiler.profile(with_flops=True, activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA]) as prof:
                start_time = time.time()
                _ = trainer.model(**inputs)
                end_time = time.time()

            runtime += (end_time - start_time)
            flops += sum(k.flops for k in prof.key_averages()) / 1e9 # Convert Flops to GFLOPs

    avg_runtime = runtime / 10
    avg_flops = flops / 10
    
    return avg_runtime, avg_flops, flops, runtime

In [ ]:
def compute_metrics_wrapper(mod_name, comp):
    def inner(eval_pred):
        return compute_metrics(eval_pred, lang, labels[lang], mod_name, comp)
    return inner

In [12]:
max_avg_runtime = 5
max_avg_flops = 5000

def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
            0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
            0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

In [13]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # for CUDA devices
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable to ensure deterministic operations

In [ ]:
# Training the models for each language and saving the the models and thier
# tokenizers after training.

# trying different pre-trained models such as roberta-base, distilbert-base, or codebert-large

seed = 27
set_seed(seed)

learning_rates = [5e-4, 5e-5, 5e-6]
epochs = [5, 10, 15, 20]
batch_sizes = [4, 8, 16]
weight_of_decay = [0.01, 0.001]
gs_metrics_dict = {}
final_model_stats = {}

for lr in learning_rates:
  for wd in weight_of_decay:
    for epoch in epochs:
      for bs in batch_sizes:
        print(f'------------------ Starting model ==> epochs: {epoch}, batch size: {bs}, weights of decay: {wd} ---------------------')
        fin_mod_name = f'epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'
        total_flops = 0
        total_time = 0
        total_avg_runtime = 0
        total_avg_flops = 0

        for lang in langs:

          lang_mod_name = f'{lang}_epoch-{epoch}_batchSize-{bs}_weightsOfDecay-{wd}'
          wandb.init(
              project="NBSE2025_GridSearch_3", 
              entity="ayoungren-colostate",
              name=lang_mod_name,
              config={  # Log hyperparameters for each run
                  "epochs": epoch,
                  "batch_size": bs,
                  "weight_decay": wd,
                  "language": lang
              }
          )
          
          num_labels = len(labels[lang])
          model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, problem_type="multi_label_classification")

          dataset = preprocess_dataset(ds[f'{lang}_train'])
          train_valalidation_split = dataset.train_test_split(test_size=0.2, seed=seed)

          train_dataset =  train_valalidation_split['train']
          validation_dataset = train_valalidation_split['test']

          training_args = TrainingArguments(
            output_dir=f'./results_{lang_mod_name}',
            eval_strategy="epoch",
            save_strategy="epoch",
            logging_dir=f'./logs_{lang_mod_name}',
            per_device_train_batch_size=bs,
            per_device_eval_batch_size=bs,
            num_train_epochs=epoch,
            weight_decay=wd,
            learning_rate=lr,
            logging_steps=1000,
            save_total_limit=2,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            seed=seed
          )
          
          trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset= validation_dataset,
            processing_class=tokenizer,
            compute_metrics=compute_metrics_wrapper(fin_mod_name, 'validation'),
          )

          trainer.train()
          metrics = trainer.evaluate()

          print("Evaluation Metrics:", metrics)

          avg_runtime, avg_flops, total_flops, total_time = measure_runtime_and_flops(trainer, validation_dataset)

            # store each model and succession score by individual language
          lang_mod_scores = scores[(scores['lan'] == lang) & (scores['model'] == fin_mod_name)]
          print('CHECK SCORE', scores[(scores['lan'] == lang) & (scores['model'] == fin_mod_name)])
          lang_avg_f1 = lang_mod_scores['f1'].mean()

          sc = round(score(lang_avg_f1, avg_runtime, avg_flops), 2)
          gs_metrics_dict[lang_mod_name] = {
            'lang': lang,
            'model': fin_mod_name,
            'avg_runtime': avg_runtime,
            'avg_flops': avg_flops,
            'total_flops': total_flops,
            'total_time': total_time,
            'avg_f1': lang_avg_f1,
            'succ_score': sc
          }

          total_avg_runtime += avg_runtime
          total_avg_flops += avg_flops

          trainer.model.save_pretrained(f'./models/{lang_mod_name}')
          tokenizer.save_pretrained(f'./tokenizers/{lang_mod_name}')
          
        print(scores)
        # store combined average succession score of all languages trained on. (Average Combined score of Java, Python, and Pharo)
        model_stats = scores[scores['model'] == fin_mod_name]
        fin_avg_f1 = model_stats['f1'].mean()
        final_model_stats[fin_mod_name] = round(score(fin_avg_f1, total_avg_runtime, total_avg_flops), 2)

------------------ Starting model ==> epochs: 5, batch size: 4, weights of decay: 0.01 ---------------------


c:\Users\ayoun\Miniconda3\envs\nbse2025\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 7614/7614 [00:00<00:00, 18059.01 examples/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a differen

{'loss': 0.161, 'grad_norm': 0.2385871857404709, 'learning_rate': 4.343401181877873e-05, 'epoch': 0.66}


 20%|██        | 1523/7615 [01:37<06:15, 16.21it/s]C:\Users\ayoun\AppData\Local\Temp\ipykernel_22788\3590710138.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores = pd.concat([scores, temp_scores], ignore_index=True)
                                                   
 20%|██        | 1523/7615 [01:42<06:15, 16.21it/s]


 ** Evaluating metrics for java test set. ** 

{'eval_loss': 0.14922671020030975, 'eval_f1': 0.5111289893021813, 'eval_precision': 0.5001785203458973, 'eval_recall': 0.5258611714970002, 'eval_runtime': 5.5167, 'eval_samples_per_second': 276.073, 'eval_steps_per_second': 69.064, 'epoch': 1.0}


...

100%|██████████| 15240/15240 [21:14<00:00, 11.96it/s]


{'train_runtime': 1274.6978, 'train_samples_per_second': 95.568, 'train_steps_per_second': 11.956, 'train_loss': 0.024718011516755022, 'epoch': 20.0}


100%|██████████| 191/191 [00:03<00:00, 50.23it/s]



 ** Evaluating metrics for java test set. ** 

Evaluation Metrics: {'eval_loss': 0.1282494068145752, 'eval_f1': 0.8863374424065347, 'eval_precision': 0.8947534470804962, 'eval_recall': 0.8799501744697215, 'eval_runtime': 3.8528, 'eval_samples_per_second': 395.294, 'eval_steps_per_second': 49.574, 'epoch': 20.0}


SafetensorError: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })

Evaluating the predictions.

In [15]:
scores

,comparison,model,lan,cat,precision,recall,f1
0,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,python,Usage,0.897959,0.727273,0.803653
1,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,python,Parameters,0.852174,0.875000,0.863436
2,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,python,DevelopmentNotes,0.681818,0.750000,0.714286
3,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,python,Expand,0.657895,0.724638,0.689655
4,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,python,Summary,0.740741,0.821918,0.779221
5,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,pharo,Keyimplementationpoints,0.815789,0.738095,0.775000
6,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,pharo,Example,0.926316,0.871287,0.897959
7,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,pharo,Responsibilities,0.711111,0.640000,0.673684
8,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,pharo,Classreferences,1.000000,0.250000,0.400000
9,validation,epoch-20_batchSize-4_weightsOfDecay-0.01,pharo,Intent,0.939394,0.815789,0.873239


In [16]:
best_f1_scores = []
best_f1_java_row = scores[scores['lan'] == 'java'].loc[scores[scores['lan'] == 'java']['f1'].idxmax()]
best_f1_scores.append(best_f1_java_row)

best_f1_python_row = scores[scores['lan'] == 'python'].loc[scores[scores['lan'] == 'python']['f1'].idxmax()]
best_f1_scores.append(best_f1_python_row)

best_f1_pharo_row = scores[scores['lan'] == 'pharo'].loc[scores[scores['lan'] == 'pharo']['f1'].idxmax()]
best_f1_scores.append(best_f1_pharo_row)
best_f1_scores

[comparison                                  validation
 model         epoch-20_batchSize-8_weightsOfDecay-0.01
 lan                                               java
 cat                                          Ownership
 precision                                     0.947368
 recall                                        0.981818
 f1                                            0.964286
 Name: 13, dtype: object,
 comparison                                  validation
 model         epoch-20_batchSize-4_weightsOfDecay-0.01
 lan                                             python
 cat                                         Parameters
 precision                                     0.852174
 recall                                           0.875
 f1                                            0.863436
 Name: 1, dtype: object,
 comparison                                  validation
 model         epoch-20_batchSize-4_weightsOfDecay-0.01
 lan                                              pha

In [21]:
gs_metrics_dict

{'java_epoch-5_batchSize-4_weightsOfDecay-0.01': {'lang': 'java',
  'model': 'epoch-5_batchSize-4_weightsOfDecay-0.01',
  'avg_runtime': 77.52191851139068,
  'avg_flops': 34044.01424896312,
  'total_flops': 340440.1424896312,
  'total_time': 775.2191851139069,
  'avg_f1': nan,
  'succ_score': nan},
 'python_epoch-5_batchSize-4_weightsOfDecay-0.01': {'lang': 'python',
  'model': 'epoch-5_batchSize-4_weightsOfDecay-0.01',
  'avg_runtime': 19.481082320213318,
  'avg_flops': 8427.177680895456,
  'total_flops': 84271.77680895456,
  'total_time': 194.81082320213318,
  'avg_f1': nan,
  'succ_score': nan},
 'pharo_epoch-5_batchSize-4_weightsOfDecay-0.01': {'lang': 'pharo',
  'model': 'epoch-5_batchSize-4_weightsOfDecay-0.01',
  'avg_runtime': 13.52945659160614,
  'avg_flops': 5811.847475200231,
  'total_flops': 58118.47475200231,
  'total_time': 135.2945659160614,
  'avg_f1': nan,
  'succ_score': nan},
 'java_epoch-5_batchSize-8_weightsOfDecay-0.01': {'lang': 'java',
  'model': 'epoch-5_batchS

In [22]:
final_model_stats

{'epoch-5_batchSize-4_weightsOfDecay-0.01': -5.51,
 'epoch-5_batchSize-8_weightsOfDecay-0.01': -5.67,
 'epoch-10_batchSize-4_weightsOfDecay-0.01': -5.8,
 'epoch-10_batchSize-8_weightsOfDecay-0.01': -5.95,
 'epoch-15_batchSize-4_weightsOfDecay-0.01': -6.02,
 'epoch-15_batchSize-8_weightsOfDecay-0.01': -6.05,
 'epoch-20_batchSize-4_weightsOfDecay-0.01': -6.07}

In [20]:
best_fin_succ_score = max(final_model_stats.values())
best_fin_succ_score

-5.51